In [1]:
import torch
from torch import nn

In [51]:
ln = nn.LayerNorm(64)
def ttt_inner_1_manual(W0_DxD, Q_TxD, K_TxD, V_TxD):
    # y = ln(z), z = KW
    Z_TxD = K_TxD @ W0_DxD
    dy = ln(Z_TxD) - V_TxD
    _, dz = torch.autograd.functional.vjp(lambda x: ln(x), Z_TxD, dy)
    return torch.tril(Q_TxD @ K_TxD.T) @ (-dz)

def ttt_inner_1_auto(W0_DxD, Q_TxD, K_TxD, V_TxD):
    Ys = []
    for i in range(Q_TxD.shape[0]):
        loss_fn = lambda x_DxD: 0.5 * torch.sum((ln(K_TxD[:i+1] @ x_DxD) - V_TxD[:i+1]) ** 2)
        dw_i = torch.func.grad(loss_fn)(W0_DxD)
        Ys.append(Q_TxD[i] @ (-dw_i))
    return torch.stack(Ys)

In [52]:
D = 64
X_TxD = torch.randn((100, D), requires_grad=True)
QKVproj = nn.Linear(D, 3 * D)
QKV_Tx3D = QKVproj(X_TxD)
Q_TxD, K_TxD, V_TxD = torch.split(QKV_Tx3D, D, -1)

In [54]:
W0_DxD = torch.zeros((D, D), device=K_TxD.device)
out_auto = ttt_inner_1_auto(W0_DxD, Q_TxD, K_TxD, V_TxD)
out_manual = ttt_inner_1_manual(W0_DxD, Q_TxD, K_TxD, V_TxD)
torch.abs(out_auto - out_manual).mean().item(), torch.abs(out_auto - out_manual).max().item()

(0.0008264871430583298, 0.01171875)

In [75]:
def ttt_inner_2_manual(W0_DxD, Q_TxD, K_TxD, V_TxD):
    # y = ln(z), z = KW
    Z_TxD = K_TxD @ W0_DxD
    dy = ln(Z_TxD) - V_TxD
    dz1, = torch.func.vjp(ln, Z_TxD)[1](dy)

    Z_TxD = Z_TxD - torch.tril(K_TxD @ K_TxD.T) @ dz1
    dy = ln(Z_TxD) - V_TxD
    dz2, = torch.func.vjp(ln, Z_TxD)[1](dy)

    A_TxT = torch.tril(Q_TxD @ K_TxD.T)
    return Q_TxD @ W0_DxD + A_TxT @ (-dz1) + A_TxT @ (-dz2)

def ttt_inner_2_auto(W0_DxD, Q_TxD, K_TxD, V_TxD):
    Ys = []
    for i in range(Q_TxD.shape[0]):
        loss_fn = lambda x_DxD: 0.5 * torch.sum((ln(K_TxD[:i+1] @ x_DxD) - V_TxD[:i+1]) ** 2)
        dw_i = torch.func.grad(loss_fn)(W0_DxD)
        W1_DxD = W0_DxD - dw_i
        dw_i = torch.func.grad(loss_fn)(W1_DxD)
        W2_DxD = W1_DxD - dw_i
        Ys.append(Q_TxD[i] @ W2_DxD)
    return torch.stack(Ys)

In [76]:
W0_DxD = torch.zeros((D, D), device=K_TxD.device)
out_auto = ttt_inner_1_auto(W0_DxD, Q_TxD, K_TxD, V_TxD)
out_manual = ttt_inner_2_manual(W0_DxD, Q_TxD, K_TxD, V_TxD)
(torch.abs(out_auto - out_manual) / torch.maximum(torch.abs(out_auto), torch.abs(out_manual))).max()

tensor(0.0008, grad_fn=<MaxBackward1>)